# Exercise 1.1.1: Choosing Metrics
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives
* Implement a function that calculates [IoU](https://en.wikipedia.org/wiki/Jaccard_index) scores between two bounding boxes;
* Calculate [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) for a given set of prediction and the ground truth pairs.

## 1. Introduction


In [3]:
### Importing required modules

In [4]:
import json
import numpy as np
import pandas as pd
from typing import List

### 1.1. Calculating IoU

Before we begin our implementation, let's review the definition of IoU from [Rezatofighi et al., 2018](https://giou.stanford.edu/) [1] and why we choose this metric for our bounding box prediction task.

_Intersection over Union (IoU), also known as [Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index), is the most popular evaluation metric for tasks such as segmentation, object detection and tracking. In object detection, IoU is used as a metric to evaluate how close the prediction and ground truth bounding boxes overlap._ 

_IoU has the appealing property of scale invariance. This means that the width, height and location of the two bounding boxes under consideration are taken into account. The normalized IoU measure focuses on the area of the shapes, no matter their size._

In summary, IoU is a metric that describes the amount of overlap between two bounding boxes with a normalized score ranging from `0.0` to `1.0`. The greater the overlapping region, the greater our IoU score will be. The IoU score will be an important factor in determining the "quality" of our bounding box predictions, as the size of the overlapping regions of interest will vary with respect to the accuracy of our predicted coordinates.

Illustrated in Fig. 1 is a set of possible bounding box configuratons. For our problem, we can consider one of the two boxes to be formed from the ground truth coordinates and the other to be formed from predicted coordinates. In the case of our problem, a full overlap shown in Fig. 1(d) represents a perfect prediction and is given an IoU score of `1.0`. On the other hand, zero overlap shown in Fig. 1(a) corresponds to an IoU score of `0.0` and usually indicates a complete mismatch between the predicted and ground truth bounding boxes.


#### Possible bounding box configurations

![Fig. 1. IoU - Four possible bounding boxes.](figures/2022-07-25-Figure-1-IoU-Boxes.jpg)

$$
\begin{align}
    \textrm{Fig. 1. } & \textrm{Four possible bounding box pairs with highlighted regions of intersection (shown in black).} \\
    \end{align}
$$

#### Possible non-overlapping bounding boxes

It will be important to note in our algorithm the four possible cases when the two bounding boxes _do not_ overlap, as illustrated in Fig. 1(a). There are four possible cases when two boxes may not overlap, corresponding to the locations of the respective left and right, upper and lower edges of boxes 1 and 2. 

These four cases of non-overlap are illustrated in Fig. 2 below and will be covered in more detail in the later steps of this assignment. Note that in our problem we define "overlap" to be any pair of intersecting rectangles with a non-zero area of intersection.

![Fig. 2. IoU - Four possible bounding box overlaps.](figures/2022-07-25-Figure-2-IoU-Non-Overlapping-Boxes.jpg)

$$
\begin{align}
    \textrm{Fig. 2. } & \textrm{Four possible bounding box pairs, each with zero overlap.} \\
    \end{align}
$$

#### Regions of interest

In order to calculate IoU for our bounding box prediction problem, we need to identify several areas of interest. 

The first region we will calculate is the _area of intersection_ shown in Fig. 3(a). This is the region contained inside the overlapping predicted and ground truth bounding boxes.

The second region we will calculate is the _area of union_ shown in Fig. 3(b). This is the total combined area of both boxes. Note that when calculating this area we will subtract off the inner-most sub-region, i.e., the area of intersection, to prevent it from contributing twice in our IoU formula (more on this later).

![Fig. 3. The two overlapping areas of interest (shown in black).](figures/2022-07-25-Figure-3-IoU-Regions-of-Interest.jpg)
$$
\begin{align}
    \textrm{Fig. 3. } & \textrm{The two overlapping areas of interest (shown in black).} \\
    \end{align}
$$

Once we have these two areas computed, we can the obtain the normalised IoU score derived from the formula illustrated below. The IoU score is simply the ratio between the two areas and is naturally normalised between `0.0` and `1.0`. 


![Fig. 4. The normalised IoU score formula.](figures/2022-07-25-Figure-4-IoU-Formula.jpg)

$$\textrm{Fig. 4. The normalised IoU score formula illustrated for the bounding box prediction problem.}$$

#### Calculating area of intersection

In order to solve for the area of the inner region shown in the numerator of Fig. 4, recall the two points derived from our previous calculations above. That is, the coordinates $(x_3, y_3)$ and $(x_2, y_2)$ from Fig. 3. From these we can obtain the width, $w$, and height, $h$, of the intersection by taking the difference of the two $x$- and $y$- coordinate values,

$$
\begin{align}
    w_b & = \left \vert x_2 - x_3 \right \vert, \\
    h_b & = \left \vert y_2 - y_3 \right \vert.
    \end{align}
$$

From there we can find our _area of intersection_ value, which will serve as the numerator of the _IoU_ formula,

$$ 
\begin{align}
    Area_{i} & = w_b * h_b. \\
    \end{align}
$$

#### Calculating area of union

The denominator of the IoU formula shown in Fig. 4 corresponds to the _total_ or combined area of both bounding boxes. We will work backwards to find this by first computing each box's area individually. For the upper-left box, we have

$$
\begin{align}
    w_1 & = \left \vert x_2 - x_1 \right \vert, \\
    h_1 & = \left \vert y_2 - y_1 \right \vert, \\
    Area_1 &= w_1 * h_1.
    \end{align}
$$


Repeating for the lower-right box, we have

$$
\begin{align}
    w_2 &= \left \vert x_4 - x_3 \right \vert, \\
    h_2 &= \left \vert y_4 - y_3 \right \vert, \\
    Area_2 &= w_2 * h_2.
    \end{align}
$$

To obtain the area of union, we will then add the two area values we calculated. However, we must subtract off the area of the intersection found above since we only want to count it once,

$$ 
\begin{align}
    Area_{u} &= {Area_1 + Area_2} - {Area_i}. \\
    \end{align}
$$

Since we are interested in computing the _areas_ of _intersection_ and _union_, we are going to need a few extra calculations. Luckily, these are very easy formulas to recall from elementary geometry. As we are working with rectangle bounding boxes, the area is given by:

$$
\begin{align}
    Area &= \left \vert w * h \right \vert. \\
    \end{align}
$$

#### Calculating the IoU score
Last but not least, we will use the values we obtained above to fit the formula illustrated in Fig. 4,
$$
\begin{align}
    IoU &= \frac{\textrm{Area of Intersection}}{\textrm{Area of Union}} = \frac{Area_i}{Area_u}. \\
    \end{align}
$$

Not convinced? We will look at one additional example specific to our problem. 

In the next section, we will cover the IoU formula applied to a more specific example — the bounding box prediction problem we are trying to solve. 

Before we move on, let's recap the steps above:

1. Determine if two bounding boxes overlap;
1. If so, identify the two areas of interest — _intersection_ and _union_;
2. Compute their respective areas;
3. Obtain the IoU score by calculating the ratio of the two areas.

## 2. Programming Task

### 2.1. IoU for Bounding Box Prediction


Now that you have the IoU formula down, let's consider a more practical example applied to the bounding box prediction problem we are trying to solve.

![Fig. 5. A practical example of the IoU scoring algorithm applied to bounding box prediction for 2D object recognition.](figures/2022-07-25-Figure-5-IoU-Vehicle.jpg)

$$
\begin{align}
    \textrm{Fig. 5. } & \textrm{A starting point for the IoU scoring algorithm applied to bounding box prediction for 2D object recognition.} \\
    \end{align}
$$


Here shown in Fig. 5. we have a pair of bounding boxes labelled with their coordinate pairs. As shown in the figure legend, the solid green coloured box indicates the _ground truth_ bounding box, while the dashed red coloured box is the corresponding _predicted_ bounding box.


In order to perform the IoU calculations described above, we will need to first identify the the upper-left and lower-right coordinates highlighted in yellow above. These coordinate values will be used in the later steps of our IoU algorithm to compute the area of intersection and in turn the IoU score. Before we get into the details of this important step, let's first review our input data and several key assumptions.

#### Considerations for our input data

Let's consider the data we are given as input to our `calculate_ious` function. The first list, `gt_bboxes`, contains `(x, y)`coordinate pairs corresponding to the upper-left and lower-right coordinates of each ground truth bounding box. The second list, `pred_bboxes`, is similar to the first, but contains instead the coordinate pairs of each _predicted_ bounding box. It is then our job to figure out the overlapping region of interest between each _ground truth_ and _predicted_ bounding box, as illustrated in Fig. 5 above. To do so, we need to know which values the coordinates $(x_3, y_3)$ and $(x_2, y_2)$ represent in our input lists. While this looks like a trivial exercise from the example above, we have to acknowledge a few important assumptions before we get into calculating the IoU forumla.

#### Making key assumptions

First assumption we make relates to our coordinate reference system. We will define the origin `(0,0)` to be the upper-left most corner of our image. Following this convention, the x-axis values will increase going right and the y-axis values will increase going down (as shown in Fig. 5).

Our second assumption governs our input values in the `pred_bboxes` and `gt_bboxes` lists. Each coordinate pair is a set of two integer values `x` and `y` describing either the _upper-left_ or _lower-right_ most coordinates of each bounding box. For example, the following input lists
```
gt_bboxes = [x_1, y_1, x_2, y_2]
pred_bboxes = [x_3, y_3, x_4, y_4]
```
describe an overlap of bounding boxes illustrated in Fig. 3.

#### Obtaining the coordinates of the area of intersection
Recall from Fig. 1 that these bounding boxes can overlap in many ways than one, or not overlap at all. 

To calculate the yellow-highlighted pair of coordinates describing the area of intersection in Fig. 5, we will make use of Python's built-in `min` and `max` formulas. 

Given the coordinate reference system we defined above, we will be using the `max` function over the two boxes' upper-left `x`- and `y`-coordinate values, and the `min` function over the lower-right `x`- and `y`-coordinate values, as such:
```
(x_b1, y_b1) := max(x_1, x_3), max(y_1, y_3)
(x_b2, y_b2) := min(x_2, x_4), min(y_2, y_4)
```

```
(x_inter1, y_inter1) := max(x_1, x_3), max(y_1, y_3)
(x_inter2, y_inter2) := min(x_2, x_4), min(y_2, y_4)
```
The region of intersection is then described by the upper-left coordinates `(x_b1, y_b1)` and lower-right coordinates `(x_b2, y_b2)` which, from Fig. 3, are points `(x_3, y_3)` and `(x_2, y_2)` respectively.

#### Calculating the areas of interest

Now that we have obtained the respective coordinate pairs, we can then calculate the area of the two key regions of interest. 

Once we have these two areas computed using the formulas described in _Part 1_, we can proceed to obtain the normalised IoU score classifying the "quality" of the bounding box prediction relative to the ground truth.

#### Determining the quality of predictions
While the IoU score can be a pretty subjective measure of prediction accuracy, there are a few "rules of thumb" that interpret the IoU scores:
* "Very good": $ 0.9 \lt IoU \leq 1.0$;
* "Good": $0.7 \lt IoU \leq 0.9$;
* "Fair": $0.5 \lt IoU \leq 0.7$.

Any pair of bounding boxes with an $IoU$ score of less than $0.5$ are typically considered disjoint, i.e., boxes bounding _not_ belonging to the same object.

#### Revisiting the example in Fig. 4

Still not sure you have the IoU algorithm down? No fret — let's go through it together step-by-step using real coordinate values. It's always a great idea to go through an algorithm line-by-line using test cases like the one illustrated in Fig. 5. Let's get started!


From Fig. 5 we have two bounding boxes; the greeen solid line is our ground-truth bounding box, and the red dotted line is our predicted bounding box. Therefore, we can expect the following set of input data,
$$
\begin{align}
    \texttt{gt_bboxes} &= \texttt{[2, 3, 13, 10]} \\
    \texttt{pred_bboxes} &= \texttt{[3, 2, 13, 11]} \\
    \end{align}
$$

our goal is to then return the correspnding IoU score that assigns a sort-of accuracy metric to the bounding box prediction shown in red.

The first step of our IoU algorithm is to find the coordinates of the area of intersection. Let's start by finding the upper-left x- and y-coordinates now —
$$
\begin{align}
    \texttt{x_inter1} &= \texttt{max(gt_bboxes[0], pred_bboxes[0])} \\
                      &= \texttt{max(2, 3)} = \textbf{3}, \\
    \texttt{y_inter1} &= \texttt{max(gt_bboxes[1], pred_bboxes[1])} \\
                      &= \texttt{max(3, 2)} = \textbf{3}. \\    
    \end{align}
$$

We can verify that our result for the first coordinate pair `(x_inter1, y_inter1)` is indeed equal to `(3, 3)`, matching the highlighted upper-left coordinate pair shown in Fig. 5. 

Repeating for the lower-right x- and y-coordinates —
$$
\begin{align}
    \texttt{x_inter2} &= \texttt{min(gt_bboxes[2], pred_bboxes[2])} \\
                      &= \texttt{min(13, 13)} = \textbf{13}, \\
    \texttt{y_inter2} &= \texttt{min(gt_bboxes[3], pred_bboxes[3])} \\
                      &= \texttt{min(10, 11)} = \textbf{10}. \\    
    \end{align}
$$
Again, our second coordinate pair of `(13, 10)` checks out, matching the highlighted lower-right coordinate pair shown in Fig. 5. Awesome!


The second step in our IoU algorithm is to calculate the areas of intersection. In order to do so, we first obtain the _width_ and _height_ of the rectangular region of interesection.
$$
\begin{align}
    \texttt{w_inter} &= \texttt{abs(x_inter2 - x_inter1)} \\
                     &= \texttt{abs(13 - 3)} = \textbf{10}, \\
    \texttt{h_inter} &= \texttt{abs(y_inter2 - y_inter1)} \\
                     &= \texttt{abs(10 - 3)} = \textbf{7}, \\  
    \end{align}
$$

Proceeding to calculate the area of the intersection from these values, we obtain
$$
\begin{align}
    \texttt{area_inter} &= \texttt{w_inter * h_inter} \\
                        &= \texttt{10 * 7} = \textbf{70}. \\  
    \end{align}
$$

Through visual inspection we can see that the value of $70$ we obtained for the area of intersection indeed matches what we see in Fig. 5.

Moving on to the third step of our IoU algorithm: calculating the coordinates belonging to the area of union. We will start by fnding the _width_ and _height_ belonging to each of the two bounding boxes, then calculating their areas independently of each other —
$$
\begin{align}
    \texttt{w_union1} &= \texttt{abs(gt_bboxes[2] - gt_bboxes[0])} \\
                      &= \texttt{abs(13 - 2)} = \textbf{11}, \\
    \texttt{h_union1} &= \texttt{abs(gt_bboxes[3] - gt_bboxes[1])} \\
                      &= \texttt{abs(10 - 3)} = \textbf{7}, \\
    \texttt{area_union1} &= \texttt{w_union1 * h_union1} \\
                         &= \texttt{11 * 7} = \textbf{77}. \\  
    \end{align}
$$

The first area we obtain corresponds to the ground truth bounding box with a value of $77$. Repeating for the predicted bounding box,
$$
\begin{align}
    \texttt{w_union2} &= \texttt{abs(pred_bboxes[2] - pred_bboxes[0])} \\
                      &= \texttt{abs(13 - 3)} = \textbf{10}, \\
    \texttt{h_union2} &= \texttt{abs(pred_bboxes[3] - pred_bboxes[1])} \\
                      &= \texttt{abs(11 - 2)} = \textbf{9}, \\
    \texttt{area_union2} &= \texttt{w_union2 * h_union2} \\
                         &= \texttt{10 * 9} = \textbf{90}. \\  
    \end{align}
$$

we obtain an value of $90$. To calculate the _area of union_, we will follow the formula from Sect. 1.1 to eliminate the duplicate overlapping region and leave us with the combined area of both boxes. In Fig. 5, this is done as follows
$$
\begin{align}
    \texttt{area_union} &= \texttt{(area_union1 + area_union2) - area_inter} \\
                        &= \texttt{(77 + 90) - 70} = \textbf{97}. \\
    \end{align}
$$

Lastly, we will compute the IoU score for the bounding box pair in order to obtain a prediction accuracy metric. This is simply the ratio of the area of intersection to area of union,
$$
\begin{align}
    \texttt{IoU} &= \texttt{(area_inter) / (area_union)} \\
                 &= \texttt{(70) / (97)} \\
                 &\approx \textbf{0.72}. \\
    \end{align}
$$

Great — we have shown that our algorithm correctly computes the IoU score for the example shown in Fig. 5. Using the "rule of thumb" interpretation of the resulting IoU score of $0.72$ leaves us with a prediction accuracy classification of _good_ meaning that we can safely assume the predicted bounding box roughly encloses the true area of the vehicle.

#### IoU algorithm in Python

In [5]:
### From Udacity's `iou.py`

In [6]:
class IoU:

    def __init__(self):
        return

    @staticmethod
    def _overlapping_rectangles(bbox1: List[int], bbox2: List[int]) -> bool:
        """Returns True if the bounding boxes overlap.
        
        Two bounding boxes overlap if their area is positive and non-zero.
        
        :param bbox1: 1x4 list of x-y coordinates forming a rectangle.
        :param bbox2: 1x4 list of x-y coordinates forming a rectangle.
        :returns: bool, whether or not the two rectangles overlap.
        """
        
        # 1. Check if bounding boxes do not overlap
        if (
            # (a) First bbox lower edge is GEQ second bbox upper edge
            (min(bbox1[1], bbox2[1]) >= max(bbox1[3], bbox2[3])) or
            # (b) First bbox right edge is LEQ second bbox left edge
            (min(bbox1[2], bbox2[2]) <= max(bbox1[0], bbox2[0])) or
            # (c) First bbox left edge is GEQ second bbox right edge
            (min(bbox1[0], bbox2[0]) >= max(bbox1[2], bbox2[2])) or
            # (d) First bbox upper edge is LEQ second bbox lower edge
            (min(bbox1[3], bbox2[3]) <= max(bbox1[1], bbox2[1]))
        ):
            return False
        # 2. Check if intersection area is larger than 0
        else:
            x_inter1 = max(bbox1[0], bbox2[0])
            y_inter1 = max(bbox1[1], bbox2[1])
            x_inter2 = min(bbox1[2], bbox2[2])
            y_inter2 = min(bbox1[3], bbox2[3])
            # Overlapping region must have positive area, if not intersect, then all 0.
            w_inter = max(0, (x_inter2 - x_inter1))
            h_inter = max(0, (y_inter2 - y_inter1))
            if (w_inter * h_inter) > 0:
                return True
        return False

    def _calculate_iou(self, gt_bbox: List[int], pred_bbox: List[int]) -> float:
        """Calculates the IoU score for a single pair of bounding boxes.

        :param gt_bbox: 1x4 list of ground-truth coordinates,
        :param pred_bbox: 1x4 list of predicted coordinates,
        returns: iou, the pairwise IoU score between the two bounding boxes.
        """

        # 1. Check if bounding boxes overlap
        if not self._overlapping_rectangles(gt_bbox, pred_bbox):
            return 0.0
        else:
            # 2. Find the coordinates of the area of intersection
            #    2a. The upper-left coordinate
            x_inter1 = max(gt_bbox[0], pred_bbox[0])
            y_inter1 = max(gt_bbox[1], pred_bbox[1])
            #    2b. The lower-right coordinate
            x_inter2 = min(gt_bbox[2], pred_bbox[2])
            y_inter2 = min(gt_bbox[3], pred_bbox[3])
            # 3. Calculate the area of intersection
            w_inter = abs(x_inter2 - x_inter1)
            h_inter = abs(y_inter2 - y_inter1)
            area_inter = w_inter * h_inter
            # 4. Find the coordinates of the area of union
            #    4a. The height and width of the first box 
            w_union1 = abs(gt_bbox[2] - gt_bbox[0])
            h_union1 = abs(gt_bbox[3] - gt_bbox[1])
            #    4b. The height and width of the second box
            w_union2 = abs(pred_bbox[2] - pred_bbox[0])
            h_union2 = abs(pred_bbox[3] - pred_bbox[1])
            # 5. Calculate the area of union
            area_union = (w_union1 * h_union1) + (w_union2 * h_union2)
            area_union -= area_inter
            # 6. Calculate the resulting IoU score
            iou = float(area_inter) / float(area_union)
        return iou

    def calculate_ious(self, gt_bboxes: List[List[int]], 
                       pred_bboxes: List[List[int]]) -> List[float]:
        """Calculates the IoU scores for all bounding box pairs.

        :param gt_bboxes: Nx4 list of ground-truth coordinates,
        :param pred_bboxes: Mx4 list of predicted coordinates,
        returns: iou, NxM list of pairwise IoU scores.
        """

        # Allocate a NumPy array of zeros to store the IoU scores
        ious = np.zeros((gt_bboxes.shape[0], pred_bboxes.shape[0]))
        # For each ground-truth bounding box
        for i, gt_bbox in enumerate(gt_bboxes):
            # Calculate the IoU score w.r.t the entire inference set
            for j, pred_bbox in enumerate(pred_bboxes):
                ious[i,j] = self._calculate_iou(gt_bbox, pred_bbox)
        return ious

In [7]:
### From Udacity's `solution/iou.py`

In [8]:
class IoUSolution:

    def __init__(self):
        return

    @staticmethod    
    def _overlapping_rectangles(bbox1: List[int], bbox2: List[int]) -> bool:
        """Returns True if the bounding boxes overlap.
        
        Two bounding boxes overlap if their area is positive and non-zero.
        
        :param bbox1: 1x4 list of x-y coordinates forming a rectangle.
        :param bbox2: 1x4 list of x-y coordinates forming a rectangle.
        :returns: bool, whether or not the two rectangles overlap.
        """
        
        # 1. Check if bounding boxes do not overlap (not required in solution)
        if (
            # (a) First bbox lower edge is GEQ second bbox upper edge
            (min(bbox1[1], bbox2[1]) >= max(bbox1[3], bbox2[3])) or
            # (b) First bbox right edge is LEQ second bbox left edge
            (min(bbox1[2], bbox2[2]) <= max(bbox1[0], bbox2[0])) or
            # (c) First bbox left edge is GEQ second bbox right edge
            (min(bbox1[0], bbox2[0]) >= max(bbox1[2], bbox2[2])) or
            # (d) First bbox upper edge is LEQ second bbox lower edge
            (min(bbox1[3], bbox2[3]) <= max(bbox1[1], bbox2[1]))
        ):
            return False
        # 2. Check if intersection area is larger than 0
        else:
            x_inter1 = max(bbox1[0], bbox2[0])
            y_inter1 = max(bbox1[1], bbox2[1])
            x_inter2 = min(bbox1[2], bbox2[2])
            y_inter2 = min(bbox1[3], bbox2[3])
            # Overlapping region must have positive area
            w_inter = max(0, (x_inter2 - x_inter1))
            h_inter = max(0, (y_inter2 - y_inter1))
            if (w_inter * h_inter) > 0:
                return True
        return False

    def _calculate_iou(self, gt_bbox: List[int], pred_bbox: List[int]) -> float:
        """Calculates the IoU score for a single pair of bounding boxes.

        :param gt_bbox: 1x4 list of ground-truth coordinates,
        :param pred_bbox: 1x4 list of predicted coordinates,
        returns: iou, the pairwise IoU score between the two bounding boxes.
        """

        # 1. Check if bounding boxes overlap
        if not self._overlapping_rectangles(gt_bbox, pred_bbox):
            return 0.0
        else:
            # 2. Find the coordinates of the area of intersection
            #    2a. The upper-left coordinate
            x_inter1 = max(gt_bbox[0], pred_bbox[0])
            y_inter1 = max(gt_bbox[1], pred_bbox[1])
            #    2b. The lower-right coordinate
            x_inter2 = min(gt_bbox[2], pred_bbox[2])
            y_inter2 = min(gt_bbox[3], pred_bbox[3])
            # 3. Calculate the area of intersection
            # Fixed typo in solution: erroneously adding +1 to width/height
            w_inter = max(0, x_inter2 - x_inter1)
            h_inter = max(0, y_inter2 - y_inter1)
            area_inter = w_inter * h_inter
            # 4. Find the coordinates of the area of union
            #    4a. The height and width of the first (ground truth) box
            w_union1 = gt_bbox[2] - gt_bbox[0]        # abs() not used in sol.
            h_union1  = gt_bbox[3] - gt_bbox[1]
            #    4b. The height and width of the second (predicted) box
            w_union2 = pred_bbox[2] - pred_bbox[0]    # abs() not used in sol.
            h_union2 = pred_bbox[3] - pred_bbox[1]
            # 5. Calculate the area of union
            area_union = (w_union1 * h_union1) + (w_union2 * h_union2)
            area_union -= area_inter
            # 6. Calculate the resulting IoU score
            iou = float(area_inter) / float(area_union)
        return iou

    def calculate_ious(self, gt_bboxes: List[List[int]], 
                       pred_bboxes: List[List[int]]) -> List[float]:
        """Calculates the IoU scores for all bounding box pairs.

        :param gt_bboxes: Nx4 list of ground-truth coordinates,
        :param pred_bboxes: Mx4 list of predicted coordinates,
        returns: iou, NxM list of pairwise IoU scores.
        """

        # Allocate a NumPy array of zeros to store the IoU scores
        ious = np.zeros((gt_bboxes.shape[0], pred_bboxes.shape[0]))
        # For each ground-truth bounding box
        for i, gt_bbox in enumerate(gt_bboxes):
            # Calculate the IoU score w.r.t the entire inference set
            for j, pred_bbox in enumerate(pred_bboxes):
                ious[i,j] = self._calculate_iou(gt_bbox, pred_bbox)
        return ious

#### Collecting our data

In [9]:
### From Udacity's `utils.py`

In [10]:
def get_data():
    """Simple wrapper function to get data."""

    with open('data/ground_truth.json') as f:
        ground_truth = json.load(f)
    with open('data/predictions.json') as f:
        predictions = json.load(f)
    return ground_truth, predictions

In [11]:
ground_truth, predictions = get_data()

In [12]:
### From Udacity's `iou.py`

In [13]:
# get bboxes array
filename = 'segment-1231623110026745648_480_000_500_000_with_camera_labels_38.png'
gt_bboxes = [g['boxes'] for g in ground_truth if g['filename'] == filename][0]
gt_bboxes = np.array(gt_bboxes)
gt_classes = [g['classes'] for g in ground_truth if g['filename'] == filename][0]


pred_bboxes = [p['boxes'] for p in predictions if p['filename'] == filename][0]
# Fixing typo in solution
pred_bboxes = np.array(pred_bboxes) # pred_boxes -> pred_bboxes
pred_classes = [p['classes'] for p in predictions if p['filename'] == filename][0]

#### Obtaining IoU scores

In [14]:
### Comparing results of my algorithm to provided solution

In [15]:
### Testing my IoU algorithm
ious = IoU().calculate_ious(gt_bboxes, pred_bboxes) # pred_boxes -> pred_bboxes
ious

array([[0.84313051, 0.        , 0.        , 0.        , 0.23860974],
       [0.        , 0.08469791, 0.4243356 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.73221757, 0.        ],
       [0.        , 0.41277874, 0.83450504, 0.        , 0.        ],
       [0.        , 0.68758782, 0.43810509, 0.        , 0.        ],
       [0.12221933, 0.        , 0.        , 0.        , 0.66359447],
       [0.02888778, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.02499868, 0.        , 0.        ]])

In [16]:
### Testing Udacity's IoU algorithm
ious_sol = IoUSolution().calculate_ious(gt_bboxes, pred_bboxes)
ious_sol

array([[0.84313051, 0.        , 0.        , 0.        , 0.23860974],
       [0.        , 0.08469791, 0.4243356 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.73221757, 0.        ],
       [0.        , 0.41277874, 0.83450504, 0.        , 0.        ],
       [0.        , 0.68758782, 0.43810509, 0.        , 0.        ],
       [0.12221933, 0.        , 0.        , 0.        , 0.66359447],
       [0.02888778, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.02499868, 0.        , 0.        ]])

#### Evaluating IoU score results

In [17]:
### From Udacity's `utils.py`

In [18]:
def check_results(ious):
    """Checks the IoU prediction results."""

    solution = np.load('data/exercise1_check.npy')
    print((ious == solution).sum())
    assert (ious == solution).sum() == 40, 'The iou calculation is wrong!'
    print('Congrats, the iou calculation is correct!')

In [19]:
check_results(ious)

40
Congrats, the iou calculation is correct!


### 2.2. Precision and Recall for Bounding Box Prediction
In order to evaluate the performance of our Intersection over Union (IoU) function, we will look at two very popular classification metrics in Machine Learning: [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall). In a binary classification task, precision and recall are measures of relevance. 

_Precision_ is the ratio of correct predictions to total number of predictions made, i.e., the true positives. In an information retrieval context, _precision_ addresses the question: _How many retrieved items are relevant?_.
* $Precision = \frac{TP}{TP + FP}$

In a binary classification task, a perfect precision score of `1.0` means that every predicted label of a respective class was correct.

_Recall_ is the ratio of correct predictions to total number of labels in the positive class. In an information retrieval context, _recall_ adresses the question: _How many relevant items are retrieved?_
* $Recall = \frac{TP}{TP + FN}$

In a binary classification task, a perfect recall score of `1.0` means that every item from a respective class was predicted correctly, i.e., labelled as belonging to that class.

#### Considerations for our problem
_Precision_ and _recall_ are often defined for binary classification problems involving a set of "correct" and "incorrect" class label predictions. In such problems, the true positive (_TP_) and true negative (_TN_) values are simply the number of _correct_ positive and negative class label predictions. It follows that the false positive (_FP_) and false negative (_FN_) values are the number of _incorrect_ class label predictions.

In the context of object recognition, we must adapt these definitions to fit the pairwise continuous IoU scores we computed earlier. In order to count the number of _correct_ and _incorrect_ bounding box predictions, we will use a _threshold value_ of $0.5$. That is, any IoU score above the threshold value of $0.5$ will be considered _correct_, and any IoU score below the threshold value will be considered _incorrect_. 

One other important consideration specific to the bounding box prediction task is the _elimination of the TN term_. A non-zero true negative (TN) value would indicate that our model has sometimes predicted "no bounding box" in an image that does in fact _not_ contain a bounding box. Since in our ground truth dataset we are assured that every sample contains a single bounding box, we know that a non-zero TN value could not be possible and that "no bounding box" is not a prediction our model makes. In summary, every sample in `pred_bboxes` corresponds to a predicted bounding box based on its ground truth sample which is gauranteed to contain a bounding box somewhere in the image frame.
* **True Positives (TP)**: set of bounding box predictions an IoU score greater than the threshold value (0.5) where the ground truth and predicted class labels are the same;
    * `the image contains a bounding box of class i and model predicts bounding box for class i with IoU > 0.5`.
* **False Positives (FP)**: set of bounding box predictions with an IoU score greater than the threshold value (0.5) where the predicted and ground truth class labels are _NOT_ the same;
    * `the image contains a bounding box of class i and model predicts bounding box for class j with IoU > 0.5.`
* **True Negative (TN):** set of bounding box pairs whose ground truth and predicted bounding boxes are 'None';
    * `not applicable in our dataset.`
* **False Negative (FN):** set of ground truth bounding boxes where the corresponding class predictions have IoU scores lower than the threshold.
    * `the image contains a bounding box of class i and model predicts no bounding box (IoU = 0.0) or bounding box for class i with low confidence (IoU < 0.5).`

Based on the extended definition above, we can now safely determine the _precision_ and _recall_ scores for our entire set of pairwise IoUs.


#### Classes in our dataset
Note that in our bounding box dataset, we have two class labels `1` and `2`. We are provided a list of ground truth class labels, `gt_classes`, and a list of predicted class labels, `pred_classes`, as follows
```
gt_classes = [1, 1, 1, 1, 2, 1, 1, 1]
pred_classes = [1, 2, 1, 2, 1]

```
In our IoU scoring algorithm, we computed an `iou` score for each ground truth bounding box against the set of predicted boundng boxes in a row-column wise matrix, i.e., each `[row, col]` pair in our `ious` matrix corresponds to a ground truth-prediction bounding box pair. We can visualise this a bit easier below when turning the 2D matrix into a [pandas `DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) object.

#### Revisiting the example in Fig. 4

In Fig. 4 we have a single pair of bounding boxes with a resulting IoU score of $0.72$. Since this confidence value is above our threshold of $0.5$, we will count the bounding box prediction in our set of true positives.

#### Precision and recall algorithm in Python

In [20]:
### From Udacity's `precision_recall.py`

In [31]:
class PrecisionRecall:

    def __init__(self):
        return

    def _compute_class_metrics(self, ious: List[float], gt_classes: List[int],
                               pred_classes: List[int], iou_threshold: float) -> List[dict]:
        """Computes the classification metrics for a multi-class dataset.
        
        Dataset contains pairwse IoU scores for the bounding box prediction problem.
        
        :param ious: NxM list of pairwise IoU scores
        :param gt_classes: 1xN list of ground truth class labels
        :param pred_classes: 1xM list of predicted class labels
        :param iou_threshold: float threshold, predictions 'valid' above this value.
        :returns: list of dict objects containing the per-class metrics.
        """
        
        # Store per-class metrics
        cls_metrics = []
        # Convert matrix into pandas DataFrame for easier slicing/indexing
        df = pd.DataFrame(data=ious, index=gt_classes, columns=pred_classes)
        # Compute per-class metrics
        for cls in np.unique(gt_classes):
            # Get all preds for gt cls label
            cls_df = df.loc[[cls],:]    # Preserves DataFrame structure
            print(cls_df)
            # Get all pairs with matching class labels
            cls_data = cls_df[cls]
            print(cls_data)
            # Count number of TN (i.e., no bounding box to predict, N/A for our data)
            true_negatives = 0
            # Count number of TP (correct class, IoU > 0.5) predictions
            true_positives = np.count_nonzero(cls_data.where(cls_data > 0.5).fillna(0))
            # Count number of FN (incorrect class, IoU < 0.5) predictions
            false_negatives = np.count_nonzero(cls_df.where(cls_df < 0.5).drop(columns=cls).fillna(0))
            # Count number of FP (incorrect class, IoU > 0.5) predictions
            false_positives = np.count_nonzero(cls_df.where(cls_df > 0.5).drop(columns=cls).fillna(0))
            # Calculate precsion/recall for each class and store in dict
            cls_dict = {'TN': true_negatives,
                        'TP': true_positives,
                        'FN': false_negatives,
                        'FP': false_positives
                        }
            cls_metrics.append(cls_dict)
        return cls_metrics

    def precision_recall(self, ious: List[float], gt_classes: List[int], 
                         pred_classes: List[int], iou_threshold:float=0.5) -> (float, float):
        """Calculates the precision and recall metrics.
        
        Dataset contains pairwse IoU scores for the bounding box prediction problem.
        Columns are the predicted class labels, rows are the ground truth class labels.
        
        :param ious: NxM list of pairwise IoU scores
        :param gt_classes: 1xN list of ground truth class labels
        :param pred_classes: 1xM list of predicted class labels
        :param iou_threshold: float threshold, predictions 'valid' above this value.
        :returns: (precision, recall), the two classification metric values.
        """

        # Compute per-class metrics
        cls_metrics = self._compute_class_metrics(ious, gt_classes, pred_classes, iou_threshold)
        # Store total metrics
        TN_total, TP_total = 0, 0
        FN_total, FP_total = 0, 0
        for c in cls_metrics:
            TN_total += c['TN']
            TP_total += c['TP']
            FN_total += c['FN']
            FP_total += c['FP']
        # Print total metrics
        print("TP:", TP_total, "TN:", TN_total, "\nFP:", FP_total, "FN:", FN_total)
        # Compute combined metrics
        precision = TP_total / float(TP_total + FP_total)
        recall = TP_total / float(TP_total + FN_total)
        return precision, recall

In [22]:
### From Udacity's `solution/precision_recall.py`

In [37]:
class PrecisionRecallSolution:

    def __init__(self):
        return

    def _compute_metrics(self, ious: List[float], gt_classes: List[int], 
                         pred_classes: List[int], iou_threshold: float) -> List[dict]:
        """Computes the dataset-specific classification metrics.
        
        Dataset contains pairwse IoU scores for the bounding box prediction problem.
        
        :param ious: NxM list of pairwise IoU scores
        :param gt_classes: 1xN list of ground truth class labels
        :param pred_classes: 1xM list of predicted class labels
        :param iou_threshold: float threshold, predictions 'valid' above this value.
        :returns: list of dict objects containing the per-class metrics.
        """
        
        # Store total metrics
        metrics = []
        # Store true positives/false positives
        TP = 0
        FP = 0
        # Get all IoU values above threshold
        xs, ys = np.where(ious > iou_threshold)
        print(xs, ys)
        # Go through all (row, col) matrix elements above threshold
        for x, y in zip(xs, ys):
            # Get corresponding class labels for each pairwise IoU
            if gt_classes[x] == pred_classes[y]:
                # If class labels match and above IoU threshold
                TP += 1
            else:
                # If class labels do not match and above IoU threshold
                FP += 1
        # Get number of bbox pairs above IoU threshold with matching labels
        matched_gt = len(np.unique(xs)) # use unique to avoid one ground truth matched to multiple preds.
        # Get number of bbox pairs wth mismatched labels / IoU below threshold
        FN = len(gt_classes) - matched_gt
        # Get number of gt samples with no bounding box
        TN = 0
        # Store results in dict, return dict in list
        return [{'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN}]

    def precision_recall(self, ious: List[float], gt_classes: List[int],
                         pred_classes: List[int], iou_threshold:float=0.5) -> (float, float):
        """Calculates the precision and recall metrics.
        
        Dataset contains pairwse IoU scores for the bounding box prediction problem.
        Columns are the predicted class labels, rows are the ground truth class labels.
        
        :param ious: NxM list of pairwise IoU scores
        :param gt_classes: 1xN list of ground truth class labels
        :param pred_classes: 1xM list of predicted class labels
        :param iou_threshold: float threshold, predictions 'valid' above this value.
        :returns: (precision, recall), the two classification metric values.
        """

        # Compute metrics
        cls_metrics = self._compute_metrics(ious, gt_classes, pred_classes, iou_threshold)
        # Store total metrics
        TN_total, TP_total = 0, 0
        FN_total, FP_total = 0, 0
        for c in cls_metrics:
            TN_total += c['TN']
            TP_total += c['TP']
            FN_total += c['FN']
            FP_total += c['FP']
        # Print total metrics
        print("TP:", TP_total, "TN:", TN_total, "\nFP:", FP_total, "FN:", FN_total)
        # Compute combined metrics
        precision = TP_total / float(TP_total + FP_total)
        recall = TP_total / float(TP_total + FN_total)
        return precision, recall

#### Obtaining precision and recall scores

In [24]:
### Visualising our pairwise IoU scores in a Pandas DataFrame

In [25]:
df = pd.DataFrame(data=ious, index=gt_classes, columns=pred_classes)
df

,1,2,1,2,1
1,0.843131,0.000000,0.000000,0.000000,0.238610
1,0.000000,0.084698,0.424336,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.732218,0.000000
1,0.000000,0.412779,0.834505,0.000000,0.000000
2,0.000000,0.687588,0.438105,0.000000,0.000000
1,0.122219,0.000000,0.000000,0.000000,0.663594
1,0.028888,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.024999,0.000000,0.000000


In [34]:
### Testing my Precision/Recall algorithm
precision, recall = PrecisionRecall().precision_recall(ious, gt_classes, pred_classes)

          1         2         1         2         1
1  0.843131  0.000000  0.000000  0.000000  0.238610
1  0.000000  0.084698  0.424336  0.000000  0.000000
1  0.000000  0.000000  0.000000  0.732218  0.000000
1  0.000000  0.412779  0.834505  0.000000  0.000000
1  0.122219  0.000000  0.000000  0.000000  0.663594
1  0.028888  0.000000  0.000000  0.000000  0.000000
1  0.000000  0.000000  0.024999  0.000000  0.000000
          1         1         1
1  0.843131  0.000000  0.238610
1  0.000000  0.424336  0.000000
1  0.000000  0.000000  0.000000
1  0.000000  0.834505  0.000000
1  0.122219  0.000000  0.663594
1  0.028888  0.000000  0.000000
1  0.000000  0.024999  0.000000
     1         2         1    2    1
2  0.0  0.687588  0.438105  0.0  0.0
          2    2
2  0.687588  0.0
TP: 4 TN: 0 
FP: 1 FN: 3


In [27]:
print("Precision:", precision, "\nRecall:", recall)

Precision: 0.8 
Recall: 0.5714285714285714


In [38]:
### Testing the Udacity Precision/Recall solution
precision_sol, recall_sol = PrecisionRecallSolution().precision_recall(ious_sol, gt_classes, pred_classes)

[0 2 3 4 5] [0 3 2 1 4]
TP: 4 TN: 0 
FP: 1 FN: 3


In [29]:
print("Precision:", precision_sol, "\nRecall:", recall_sol)

Precision: 0.8 
Recall: 0.5714285714285714


From this comparison we see that my `PrecisionRecall` algorithm correctly computed the precision and recall metrics according to the solution provided by Udacity. While Udacity's algorithm is simplistic, my algorithm is both readable and able to scale for multi-class datasets. This will be a useful approach when calculating average precision for the $mAP$ metric.

### 2.3. Mean Average Precision (mAP) for Bounding Box Prediction (optional)
Since we are given a dataset containing multiple class labels, i.e., bounding boxes belonging to a respective object type. While calculating mAP is not a requirement in this exercise, it is generally a good metric to observe in practice. The mAP metric can help us understand how well each classes' bounding boxes are being predicted. If you are up for the challenge, I'd encourage you to try completing this task!

#### Considerations for our problem
We are given a set of ground truth values which correspond to the true coordinates of each bounding box in our dataset. We made full use of this data in order to score the prediction set and obtain precision/recall values. We are also given a class label for each _ground truth_ and _predicted_ bounding boxes.

In [30]:
### Exercise left to the reader, will be revisited by author

## 3. Closing Remarks

##### Alternatives to IoU
* [Generalized IoU (GIoU)](https://giou.stanford.edu/): loss function that gives weight to bounding box predictions that are "closer" to ground truth despite having zero overlap, e.g., when the standard $IoU=0.0$;

$$
\begin{align}
    GIoU &=  \frac{\left \vert A \cap B \right \vert}{\left \vert A \cup B 
    \right \vert} - \frac{\left \vert C \setminus (A \cup B) \right \vert}{\left \vert C \right \vert}. \\
    \end{align}
$$

* [Mean Average Precision (mAP)](https://towardsdatascience.com/what-is-average-precision-in-object-detection-localization-algorithms-and-how-to-calculate-it-3f330efe697b): alternative to IoU; used in popular object detection models like YOLO and Faster RCNN; takes the average precision (AP) across all classes;

$$
\begin{align}
    mAP &=  \frac{1}{N} * \sum_{i=1}^{N} AP_{i}. \\
    \end{align}
$$

##### Other purposes for IoU
IoU is a popular metric in object detection for bounding box prediction tasks. IoU serves an important role in _Non-Maxima Supression_, which is an algorithm to retain a single bounding box from a set of predictions with the greatest amount of overlap (in order to discard the rest). IoU can also be used as a loss metric or cost function in many deep learning applications.

## 4. Future Work
- ✅ Visualise ground truth versus prediction bounding boxes (see [Exercise 1.1.2](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Computer-Vision/Exercises/1-1-2-Data-Acquisition-Visualisation/2022-08-01-Data-Acquisition-Visualisation.ipynb));
- ✅ Clean up precision/recall formulas, considering e.g., association matrices (see [Exercise 1.5.2](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Computer-Vision/Exercises/1-5-2-Mean-Average-Precision/2022-09-25-Mean-Average-Precision.ipynb));
- ✅ Compute per-class statistics efficiently, implementing e.g., mAP (see [Exercise 1.5.2](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Computer-Vision/Exercises/1-5-2-Mean-Average-Precision/2022-09-25-Mean-Average-Precision.ipynb)).

## Credits
This assignment was prepared by Thomas Hossler and Michael Virgo et al., Winter 2021 (link [here](https://github.com/udacity/nd013-c1-vision-starter)).

References:
* [1] _Generalized Intersection over Union_, Rezatofighi et al., The IEEE Conference on Computer Vision and Pattern Recognition (CVPR), June 2019. https://giou.stanford.edu.

Helpful resources:
* [IOU (Intersection over Union) by V.S. Subramanyam](https://medium.com/analytics-vidhya/iou-intersection-over-union-705a39e7acef)

* [Intersection over Union by A. Persson | YouTube](https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/object_detection/metrics/iou.py)

* [Rectangle Intersection Demonstration by M. Crumley](https://silentmatt.com/rectangle-intersection/)

* [Determine if two rectangles overlap each other? | Stack Overflow](https://stackoverflow.com/questions/306316/determine-if-two-rectangles-overlap-each-other)

* [Problem 836. Rectangle Overlap | LeetCode](https://leetcode.com/problems/rectangle-overlap/)

* [Practitioner's guide to IoU, Non-Max Supression, and Mean Average Precision | Medium article](https://vijayabhaskar96.medium.com/practitioners-guide-to-iou-non-max-suppression-and-mean-average-precision-e09de73a2bd8)

* [What is Average Precision in Object Detection? | Medium article](https://towardsdatascience.com/what-is-average-precision-in-object-detection-localization-algorithms-and-how-to-calculate-it-3f330efe697b)

* [One value for recall and precision for multiple classes | Udacity Knowledge forum](https://knowledge.udacity.com/questions/857563)

* [Keep selected column as DataFrame instead of Series | Stack Overflow](https://stackoverflow.com/a/56983052/8222897)